In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point
from datetime import datetime
import os
import seaborn as sns
import metpy.calc as mpcalc
import re
import calendar
from matplotlib.offsetbox import AnchoredText
sns.set()

In [ ]:
def set_data(tempo, level):

    if level == 2:
        
        data = xr.open_dataset("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/nc/975hPa_79_20.nc")
        u = data.u[:,0,:,:]                   # 'u' é a variável do dataset
    
        data = xr.open_dataset("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/nc/975hPa_79_20.nc")
        v = data.v[:,0,:,:]                   # 'v' é a variável do dataset
    
    else:
        data = xr.open_dataset("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/nc/uwnd_79_20.nc")
        u = data.u[:,0,level,:,:]                   # 'u' é a variável do dataset
    
        data = xr.open_dataset("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/nc/vwnd_79_20.nc")
        v = data.v[:,0,level,:,:]                   # 'v' é a variável do dataset
    
    time_slice = slice(tempo[0],tempo[1])
    uwnd = u.sel(time=time_slice)
    vwnd = v.sel(time=time_slice)
    
    magnitude = ((uwnd)** 2 + (vwnd)** 2) ** 0.5
    
    magSet = magnitude.groupby('time.month').mean()
    
    lons = magSet.longitude.values    #1D
    lats = magSet.latitude.values     #1D
    
    Umedia_mes = u.sel(time=time_slice).groupby('time.month').mean()
    Vmedia_mes = v.sel(time=time_slice).groupby('time.month').mean()
    
    
    return lons, lats, Umedia_mes, Vmedia_mes, magSet

In [ ]:
def figura(tempo1, tempo2, level, mes):
    
    fig, ax = plt.subplots(nrows=1,ncols=2,subplot_kw=dict(projection = ccrs.PlateCarree()), figsize=(32,26))
    
    if level == 0:
        fig.suptitle('Comparativo\nMagnitude em 700 hPA', fontsize=34, weight='bold', y = 0.75, x = 0.505)
    
    elif level ==1:
        fig.suptitle('Comparativo\nMagnitude em 850 hPA', fontsize=34, weight='bold', y = 0.75, x = 0.505)
    
    else:
        fig.suptitle('Comparativo\nMagnitude em 975 hPA', fontsize=34, weight='bold', y = 0.75, x = 0.505)
    #mes = [1,2,3,4,5,6,7,8,9,10,11,12]
    
    plott(ax[0], mes, tempo1, level)
    plott2(ax[1], mes, tempo2, level)
 
    export(tempo1 ,mes, level)

In [ ]:
def figuraComp3(tempo1, tempo2, tempo3, level, mes):
    
    fig, ax = plt.subplots(nrows=1,ncols=3,subplot_kw=dict(projection = ccrs.PlateCarree()), figsize=(32,26))

    if level == 0:
        fig.suptitle('Comparativo\nMagnitude em 700 hPA', fontsize=34, weight='bold', y = 0.75, x = 0.505)
    elif level == 1:
        fig.suptitle('Comparativo\nMagnitude em 850 hPA', fontsize=34, weight='bold', y = 0.75, x = 0.505)
    else:
        fig.suptitle('Comparativo\nMagnitude em 975 hPA', fontsize=34, weight='bold', y = 0.75, x = 0.505)
    
    #mes = [1,2,3,4,5,6,7,8,9,10,11,12]
    
    plott(ax[0], mes, tempo1, level)
    plott(ax[1], mes, tempo2, level)
    plott2(ax[2], mes, tempo3, level)
 
    #exportComp3(mes, level)

In [ ]:
def plott(ax, mes, tempo1, level):
    
    lons, lats, Umedia_mes, Vmedia_mes, magSet = set_data(tempo1, level)
    
    Uvel, lonu = add_cyclic_point(Umedia_mes.sel(month=mes), coord=lons)  #interpolando o grafico/ corrigindo a descontinuidade do meridiano
    Vvel, lonv = add_cyclic_point(Vmedia_mes.sel(month=mes), coord=lons)
   
    norm = mpl.colors.Normalize(vmin=0, vmax=12)
    
    clevs = np.arange(1,19.5,1.5)
    cf = ax.contourf(lons, lats, magSet.sel(month=mes), clevs, cmap = 'jet_r',extend='both')
    
    sp = ax.streamplot(lonu, lats, Uvel, Vvel,
                   linewidth = 1.7,
                   arrowsize = 1.2,
                   density = 2,
                   norm=norm,
                   transform=ccrs.PlateCarree())
    
    cbar = plt.colorbar(cf, orientation='horizontal', pad=0.05 , ax=ax, shrink = 1.0, aspect=40)
    cbar.ax.tick_params(labelsize=15)
    cbar.set_label("m/s", fontsize = 17, y = 0.3)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator(np.arange(-70, 50, 20)) #-70,50,20
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style = {'size': 15}
    
    ax.coastlines('50m')
    ax.set_extent([-70, 20, -50, 10])
    ax.set_title(calendar.month_abbr[mes], fontdict={'fontsize': 30}, loc = 'right')
    
    if tempo1 == per1:
    
        ax.set_title('Climatologia de 1979-01 até 2010-12', fontdict={'fontsize': 20}, loc = 'left')
        
    elif tempo1 == per4:
    
        ax.set_title('Climatologia de 2010-01 até 2018-08', fontdict={'fontsize': 20}, loc = 'left')

In [ ]:
def plott2(ax, mes, tempo3, level):
    
    lons, lats, Umedia_mes, Vmedia_mes, magSet = set_data(tempo3, level)
    
    Uvel, lonu = add_cyclic_point(Umedia_mes.sel(month=mes), coord=lons)  #interpolando o grafico/ corrigindo a descontinuidade do meridiano
    Vvel, lonv = add_cyclic_point(Vmedia_mes.sel(month=mes), coord=lons)
    
    clevs = np.arange(1,19.5,1.5)
    
    cf = ax.contourf(lons, lats, magSet.sel(month=mes), clevs, cmap = 'jet_r', extend='both')
    
    norm = mpl.colors.Normalize(vmin=0, vmax=12)
    
    #brb = ax.barbs(lonu, lats, Uvel, Vvel, magnitude, length=7, cmap = 'jet_r', norm=norm, linewidth =15) ???
    
    sp = ax.streamplot(lonu, lats, Uvel, Vvel,
                   linewidth = 1.7,
                   arrowsize = 1.2,
                   density = 2,
                   norm=norm,
                   transform=ccrs.PlateCarree())

    
    cbar = plt.colorbar(cf, orientation='horizontal', pad=0.05 , ax=ax, shrink = 1.0, aspect=40)
    cbar.ax.tick_params(labelsize=15)
    cbar.set_label("m/s", fontsize = 17, y = 0.3)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator(np.arange(-70, 50, 20)) #-70,50,20
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style = {'size': 15}
    
    ax.coastlines('50m')
    ax.set_extent([-70, 20, -50, 10])
    ax.set_title(calendar.month_abbr[mes], fontdict={'fontsize': 30}, loc = 'right')
    ax.set_title('2018 até 2020', fontdict={'fontsize': 20}, loc = 'left')

In [ ]:
    def export(tempo1, mes, level):
    
        if level == 0:
            if tempo1 == per1:

                path = calendar.month_abbr[mes]
                plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots/700hPa/compMag_700hPA_'+ calendar.month_abbr[mes]+'.png')

            if tempo1 == per4:

                path = calendar.month_abbr[mes]
                plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/comparativos/700hPa/compMag_700hPA_'+ calendar.month_abbr[mes]+'.png')

        elif level == 1:
            if tempo1 == per1:

                path = calendar.month_abbr[mes]
                plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots/850hPa/compMag_850hPA_'+ calendar.month_abbr[mes]+'.png')

            if tempo1 == per4:

                path = calendar.month_abbr[mes]
                plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/comparativos/850hPa/compMag_850hPA_'+ calendar.month_abbr[mes]+'.png')

        elif level == 2:
            if tempo1 == per1:

                path = calendar.month_abbr[mes]
                plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots/975hPa/compMag_975hPA_'+ calendar.month_abbr[mes]+'.png')

            if tempo1 == per4:

                path = calendar.month_abbr[mes]
                plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/comparativos/975hPa/compMag_975hPA_'+ calendar.month_abbr[mes]+'.png')

In [ ]:
def exportComp3(mes, level):
    
    if level == 0:
        path = calendar.month_abbr[mes]
        plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots/700hPa/comp3/compMag_975hPA_'+ calendar.month_abbr[mes]+'.png')
    
    elif level == 1:
        path = calendar.month_abbr[mes]
        plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots/850hPa/comp3/compMag_975hPA_'+ calendar.month_abbr[mes]+'.png')
    
    elif level == 2:
        path = calendar.month_abbr[mes]
        plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots/975hPa/comp3/compMag_975hPA_'+ calendar.month_abbr[mes]+'.png')

In [ ]:
per1 = ['1979-01-01','2010-12-01']     # Algumas datas a serem contempladas
per2 = ['2018-01-01','2020-01-01']
per3 = ['2010-01-01','2020-01-01']     # INUTILIZADA
per4 = ['2010-01-01','2018-08-01']

#figura(tempo1, tempo2, mes escolhido) 
    #mes escolhido = [1 até 12]
    

In [ ]:
mesPlot = np.arange(1,13)

In [ ]:
for cada in mesPlot:
    figuraComp3(per1, per4, per2, 0, cada)

In [ ]:
for cada in mesPlot:
    figuraComp3(per1, per4, per2, 1, cada)

In [ ]:
for cada in mesPlot:
    figuraComp3(per1, per4, per2, 2, cada)

In [ ]:
for cada in mesPlot:
    figura(per1, per2, 0, cada)

In [ ]:
for cada in mesPlot:
    figura(per1, per2, 1, cada)

In [ ]:
for cada in mesPlot:
    figura(per1, per2, 2, cada)

In [ ]:
for cada in mesPlot:
    figura(per4, per2, 0, cada)

In [ ]:
for cada in mesPlot:
    figura(per4, per2, 1, cada)

In [ ]:
for cada in mesPlot:
    figura(per4, per2, 2, cada)

## 12 Plots 

In [ ]:
def figura12(tempo, level):
    
    fig, ax = plt.subplots(nrows=3,ncols=4,subplot_kw=dict(projection = ccrs.PlateCarree()), figsize=(32,26))
    
    if level == 0:
        if tempo == per1:

            fig.suptitle('Magnitude em 700 hPA\n 1979-01-01 até 2010-12-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)


        if tempo == per2:

            fig.suptitle('Magnitude em 700 hPA\n 2018-01-01 até 2020-01-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)


        if tempo == per3:

            fig.suptitle('Magnitude em 700 hPA\n 2010-01-01 até 2020-01-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)


        if tempo == per4:

            fig.suptitle('Magnitude em 700 hPA\n 2010-01-01 até 2018-08-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)
    
    
    elif level == 1:
        if tempo == per1:

            fig.suptitle('Magnitude em 850 hPA\n 1979-01-01 até 2010-12-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)


        if tempo == per2:

            fig.suptitle('Magnitude em 850 hPA\n 2018-01-01 até 2020-01-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)


        if tempo == per3:

            fig.suptitle('Magnitude em 850 hPA\n 2010-01-01 até 2020-01-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)


        if tempo == per4:

            fig.suptitle('Magnitude em 850 hPA\n 2010-01-01 até 2018-08-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)
            
    
    elif level == 2:
        if tempo == per1:

            fig.suptitle('Magnitude em 975 hPA\n 1979-01-01 até 2010-12-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)


        if tempo == per2:

            fig.suptitle('Magnitude em 975 hPA\n 2018-01-01 até 2020-01-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)


        if tempo == per3:

            fig.suptitle('Magnitude em 975 hPA\n 2010-01-01 até 2020-01-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)


        if tempo == per4:

            fig.suptitle('Magnitude em 975 hPA\n 2010-01-01 até 2018-08-01', fontsize=34, weight='bold', y = 0.95, x = 0.505)
        
    
    mes = [1,2,3,4,5,6,7,8,9,10,11,12]
       
    plott12(ax[0,0], mes[0],tempo, level)      # OBS: Pra essa figura, algumas pequenas alterações são feitas na função 'plott'                                            
    plott12(ax[0,1], mes[1],tempo, level)      #      Alterações de formatação, como títulos, distanciamento do colorbar e etc...
    plott12(ax[0,2], mes[2],tempo, level)
    plott12(ax[0,3], mes[3],tempo, level)
    plott12(ax[1,0], mes[4],tempo, level)
    plott12(ax[1,1], mes[5],tempo, level)
    plott12(ax[1,2], mes[6],tempo, level)
    plott12(ax[1,3], mes[7],tempo, level)
    plott12(ax[2,0], mes[8],tempo, level)
    plott12(ax[2,1], mes[9],tempo, level)
    plott12(ax[2,2], mes[10],tempo, level)
    plott12(ax[2,3], mes[11],tempo, level)
    
    ax[0][0].set_title('LAMMOC-UFF', fontdict={'fontsize': 20}, loc = 'left')
    
    export_12(tempo, level)

In [ ]:
def plott12(ax, mes, tempo, level):
    
    lons, lats, Umedia_mes, Vmedia_mes, magSet = set_data(tempo, level)
    
    Uvel, lonu = add_cyclic_point(Umedia_mes.sel(month=mes), coord=lons)  #interpolando o grafico/ corrigindo a descontinuidade do meridiano
    Vvel, lonv = add_cyclic_point(Vmedia_mes.sel(month=mes), coord=lons)
   
    norm = mpl.colors.Normalize(vmin=0, vmax=12)
    
    clevs = np.arange(2.5,22.5,2.5)
    cf = ax.contourf(lons, lats, magSet.sel(month=mes), clevs, cmap = 'jet_r',extend='both')
    
    sp = ax.streamplot(lonu, lats, Uvel, Vvel,
                   linewidth = 1,
                   arrowsize = 0.7,
                   density = 3,
                   norm=norm,
                   transform=ccrs.PlateCarree())
    
    cbar = plt.colorbar(cf, orientation='horizontal', pad=0.1 , ax=ax, shrink = 1.0, aspect=40)
    cbar.ax.tick_params(labelsize=15)
    cbar.set_label("m/s", fontsize = 17, y = 0.3, labelpad=15)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator(np.arange(-70, 50, 20)) #-70,50,20
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style = {'size': 15}
    
    ax.coastlines('50m')
    ax.set_extent([-70, 20, -50, 10])
    ax.set_title(calendar.month_abbr[mes], fontdict={'fontsize': 30}, loc = 'right')
    
    

In [ ]:
def export_12(tempo, level):
    if level == 0:
        if tempo == per1:

            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_79_10/Nmag_700hPa_1979_2010")

        elif tempo == per2:

            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_18_19/Nmag_700hPa_2018_2020")

        elif tempo == per3:

            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_10_20/Nmag_700hPa_2010_2020")

        elif tempo == per4:
            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/Nmag_700hPa_2010_2018")
    
    elif level == 1:
        if tempo == per1:

            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_79_10/Nmag_850hPa_1979_2010")

        elif tempo == per2:

            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_18_19/Nmag_850hPa_2018_2020")

        elif tempo == per3:

            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_10_20/Nmag_850hPa_2010_2020")

        elif tempo == per4:
            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/Nmag_850hPa_2010_2018")
    
    elif level == 2: 
        if tempo == per1:

            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_79_10/Nmag_975hPa_1979_2010")

        elif tempo == per2:

            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_18_19/Nmag_975hPa_2018_2020")

        elif tempo == per3:

            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_10_20/Nmag_975hPa_2010_2020")

        elif tempo == per4:
            plt.savefig("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/Nmag_975hPa_2010_2018")
    

In [ ]:
figura12(per1,0)

In [ ]:
figura12(per1,1)

In [ ]:
figura12(per1,2)

In [ ]:
figura12(per2,0)

In [ ]:
figura12(per2,1)

In [ ]:
figura12(per2,2)

In [ ]:
figura12(per3,0)

In [ ]:
figura12(per3,1)

In [ ]:
figura12(per3,2)

In [ ]:
figura12(per4,0)

In [ ]:
figura12(per4,1)

In [ ]:
figura12(per4,2)

# Anomalia 

In [ ]:
def set_dataAnom(tempo1, tempo2, level):
    if level == 2:
        
        data = xr.open_dataset("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/nc/975hPa_79_20.nc")
        u = data.u[:,0,:,:]                   # 'u' é a variável do dataset
    
        data = xr.open_dataset("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/nc/975hPa_79_20.nc")
        v = data.v[:,0,:,:]                   # 'v' é a variável do dataset
    
    else:
        data = xr.open_dataset("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/nc/uwnd_79_20.nc")
        u = data.u[:,0,level,:,:]                   # 'u' é a variável do dataset
    
        data = xr.open_dataset("D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/nc/vwnd_79_20.nc")
        v = data.v[:,0,level,:,:]                   # 'v' é a variável do dataset
               
    time_slice1 = slice(tempo1[0],tempo1[1])
    uwnd1 = u.sel(time=time_slice1)
    vwnd1 = v.sel(time=time_slice1)
    
    magnitude1 = ((uwnd1)** 2 + (vwnd1)** 2) ** 0.5
    magSet1 = magnitude1.groupby('time.month').mean()
    
    
    time_slice2 = slice(tempo2[0],tempo2[1])
    uwnd2 = u.sel(time=time_slice2)
    vwnd2 = v.sel(time=time_slice2)
    
    magnitude2 = ((uwnd2)** 2 + (vwnd2)** 2) ** 0.5 
    magSet2 = magnitude2.groupby('time.month').mean()
    
    
    lons = magSet1.longitude.values    #1D
    lats = magSet1.latitude.values     #1D
    
    
    return lons, lats, magSet1, magSet2

In [ ]:
def figuraAnom(tempo1, tempo2, mes, level):
    
    fig, ax = plt.subplots(nrows=1,ncols=1,subplot_kw=dict(projection = ccrs.PlateCarree()), figsize=(32,26))
    if level == 0:
        fig.suptitle('Anomalia\nMagnitude em 700 hPA', fontsize=34, weight='bold', y = 0.95, x = 0.505)
    
    elif level == 1:
        fig.suptitle('Anomalia\nMagnitude em 850 hPA', fontsize=34, weight='bold', y = 0.95, x = 0.505)
        
    elif level == 2:
        fig.suptitle('Anomalia\nMagnitude em 975 hPA', fontsize=34, weight='bold', y = 0.95, x = 0.505)
  
    #mes = [1,2,3,4,5,6,7,8,9,10,11,12]
    
    plottAnom(ax, mes, tempo1, tempo2, level)
   
    exportAnom(tempo1, mes, level)

In [ ]:
def plottAnom(ax, mes, tempo1, tempo2, level):
    
    lons, lats, magSet1, magSet2 = set_dataAnom(tempo1, tempo2, level)
    
    anomalia_magnitude = magSet2.sel(month=mes) - magSet1.sel(month=mes)
    
    norm = mpl.colors.Normalize(vmin=0, vmax=12)
    
    clevs = np.arange(-6,10,0.5)
    cf = ax.contourf(lons, lats, anomalia_magnitude, clevs, cmap = 'RdBu')
    
    cbar = plt.colorbar(cf, orientation='horizontal', pad=0.1 , ax=ax, shrink = 1.0, aspect=40)
    cbar.ax.tick_params(labelsize=15)
    cbar.set_label("m/s", fontsize = 17, y = 0.3, labelpad = 15)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator(np.arange(-70, 50, 20)) #-70,50,20
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style = {'size': 15}
    
    ax.coastlines('50m')
    ax.set_extent([-70, 20, -50, 10])
    ax.set_title(calendar.month_abbr[mes], fontdict={'fontsize': 40}, loc = 'right')
    
    if tempo1 == per1:
        ax.set_title('Anomalia do período 2018-01-01 a 2020-01-01 em relação ao período de 1979-01-01 a 2010-12-01 ', fontdict={'fontsize': 26}, loc = 'left')
        
    if tempo1 == per4:
        ax.set_title('Anomalia do período 2018-01-01 a 2020-01-01 em relação ao período de 2010-01-01 a 2018-08-01 ', fontdict={'fontsize': 26}, loc = 'left')

In [ ]:
def exportAnom(tempo1, mes, level):

    if tempo1 == per1:
        if level == 0:
            path = calendar.month_abbr[mes]
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots/anomalia/700hPA/NmagnitudeAnom_700hPA_'+ calendar.month_abbr[mes]+'.png')
            
        elif level == 1:
            path = calendar.month_abbr[mes]
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots/anomalia/850hPA/NmagnitudeAnom_850hPA_'+ calendar.month_abbr[mes]+'.png')
        
        elif level == 2:
            path = calendar.month_abbr[mes]
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots/anomalia/975hPa/NmagnitudeAnom_975hPA_'+ calendar.month_abbr[mes]+'.png')
        
        
    elif tempo1 == per4:
        if level == 0:
            path = calendar.month_abbr[mes]
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/anomalia/700hPA/NmagnitudeAnom_700hPA_'+ calendar.month_abbr[mes]+'.png')
            
        elif level == 1:
            path = calendar.month_abbr[mes]
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/anomalia/850hPA/NmagnitudeAnom_850hPA_'+ calendar.month_abbr[mes]+'.png')
        
        elif level == 2:
            path = calendar.month_abbr[mes]
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/anomalia/975hPA/NmagnitudeAnom_975hPA_'+ calendar.month_abbr[mes]+'.png')

In [ ]:
for cada in mesPlot:
    figuraAnom(per1,per2,cada, 0)

In [ ]:
for cada in mesPlot:
    figuraAnom(per1,per2,cada, 1)

In [ ]:
for cada in mesPlot:
    figuraAnom(per1,per2,cada, 2)

In [ ]:
for cada in mesPlot:
    figuraAnom(per4,per2,cada, 0)

In [ ]:
for cada in mesPlot:
    figuraAnom(per4,per2,cada, 1)

In [ ]:
for cada in mesPlot:
    figuraAnom(per4,per2,cada, 2)

## 12plots

In [ ]:
def figuraAnom12(tempo1, tempo2, level):
    
    fig, ax = plt.subplots(nrows=3,ncols=4,subplot_kw=dict(projection = ccrs.PlateCarree()), figsize=(32,26))
    
    if level == 0:
        if tempo1 == per1:
            fig.suptitle('Anomalia do período 2018-01-01 a 2020-01-01 em relação ao período de 1979-01-01 a 2010-12-01\n700 hPa', fontsize=34, weight='bold', y = 0.95, x = 0.505)
    
        elif tempo1 == per4:
            fig.suptitle('Anomalia do período 2018-01-01 a 2020-01-01 em relação ao período de 2010-01-01 a 2018-08-01\n700 hPa', fontsize=34, weight='bold', y = 0.95, x = 0.505)
    
    elif level == 1:
        if tempo1 == per1:
            fig.suptitle('Anomalia do período 2018-01-01 a 2020-01-01 em relação ao período de 1979-01-01 a 2010-12-01\n850 hPa', fontsize=34, weight='bold', y = 0.95, x = 0.505)
    
        elif tempo1 == per4:
            fig.suptitle('Anomalia do período 2018-01-01 a 2020-01-01 em relação ao período de 2010-01-01 a 2018-08-01\n850 hPa', fontsize=34, weight='bold', y = 0.95, x = 0.505)
    
    elif level == 2:
        if tempo1 == per1:
            fig.suptitle('Anomalia do período 2018-01-01 a 2020-01-01 em relação ao período de 1979-01-01 a 2010-12-01\n975 hPa', fontsize=34, weight='bold', y = 0.95, x = 0.505)
    
        elif tempo1 == per4:
            fig.suptitle('Anomalia do período 2018-01-01 a 2020-01-01 em relação ao período de 2010-01-01 a 2018-08-01\n975 hPa', fontsize=34, weight='bold', y = 0.95, x = 0.505)

    mes = [1,2,3,4,5,6,7,8,9,10,11,12]
    
    
    plottAnom12(ax[0,0], mes[0],tempo1, tempo2, level)
    plottAnom12(ax[0,1], mes[1],tempo1, tempo2, level)
    plottAnom12(ax[0,2], mes[2],tempo1, tempo2, level)
    plottAnom12(ax[0,3], mes[3],tempo1, tempo2, level)
    plottAnom12(ax[1,0], mes[4],tempo1, tempo2, level)
    plottAnom12(ax[1,1], mes[5],tempo1, tempo2, level)
    plottAnom12(ax[1,2], mes[6],tempo1, tempo2, level)
    plottAnom12(ax[1,3], mes[7],tempo1, tempo2, level)
    plottAnom12(ax[2,0], mes[8],tempo1, tempo2, level)
    plottAnom12(ax[2,1], mes[9],tempo1, tempo2, level)
    plottAnom12(ax[2,2], mes[10],tempo1, tempo2, level)
    plottAnom12(ax[2,3], mes[11],tempo1, tempo2, level)
    
    ax[0][0].set_title('LAMMOC-UFF', fontdict={'fontsize': 20}, loc = 'left')
    
    exportAnom_12(tempo1, level)

In [ ]:
def plottAnom12(ax, mes, tempo1, tempo2, level):
    
    lons, lats, magSet1, magSet2 = set_dataAnom(tempo1, tempo2, level)
    
    anomalia_magnitude = magSet2.sel(month=mes) - magSet1.sel(month=mes)
    
    norm = mpl.colors.Normalize(vmin=0, vmax=12)
    
    clevs = np.arange(-6,10,0.5)
    cf = ax.contourf(lons, lats, anomalia_magnitude, clevs, cmap = 'RdBu')
    
    cbar = plt.colorbar(cf, orientation='horizontal', pad=0.1 , ax=ax, shrink = 1.0, aspect=40)
    cbar.ax.tick_params(labelsize=15)
    cbar.set_label("m/s", fontsize = 17, y = 0.3, labelpad = 15)
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=1, color='black', alpha=0.3, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_left = True
    gl.ylabels_right = False
    gl.ylines = True
    gl.xlines = True
    gl.xlocator = mticker.FixedLocator(np.arange(-70, 50, 20)) #-70,50,20
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 15}
    gl.ylabel_style = {'size': 15}
    
    ax.coastlines('50m')
    ax.set_extent([-70, 20, -50, 10])
    ax.set_title(calendar.month_abbr[mes], fontdict={'fontsize': 30}, loc = 'right')

In [ ]:
def exportAnom_12(tempo1, level):
    if level == 0:
        if tempo1 == per1:
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_anomalia/Nmagnitude12Anom_79_10_700hPa.png')

        elif tempo1 == per4:
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/anomalia/Nmagnitude12Anom_10_18_700hPa.png') 
   
    elif level == 1:
        if tempo1 == per1:
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_anomalia/Nmagnitude12Anom_79_10_850hPa.png')

        elif tempo1 == per4:
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/anomalia/Nmagnitude12Anom_10_18_850hPa.png')
    
    elif level == 2:
        if tempo1 == per1:
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/Magnitude/plots12/_anomalia/Nmagnitude12Anom_79_10_975hPa.png')

        elif tempo1 == per4:
            plt.savefig('D:/_Gabriel/Estudo/UFF/Bloqueios_Atmosfericos/Climatologias/Imagens Geradas/Vento/10_18/Magnitude/anomalia/Nmagnitude12Anom_10_18_975hPa.png')

In [ ]:
figuraAnom12(per1, per2, 0)

In [ ]:
figuraAnom12(per1, per2, 1)

In [ ]:
figuraAnom12(per1, per2, 2)

In [ ]:
figuraAnom12(per4, per2, 0)

In [ ]:
figuraAnom12(per4, per2, 1)

In [ ]:
figuraAnom12(per4, per2, 2)